# Preprocessing and Modeling

## Import required modules

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
from keras.api.layers import Dense, Embedding, GRU, LeakyReLU, Concatenate, Masking, Layer, StringLookup, Normalization, BatchNormalization
from keras.api import Input
from keras.api.models import Model
from keras.api.losses import SparseCategoricalCrossentropy
from keras.api.metrics import SparseCategoricalAccuracy, Mean, TopKCategoricalAccuracy
# from transformers.models.bert import TFBertTokenizer, TFBertEmbeddings  # embedding and tokenizer for description/nlp related stufff
from keras.api.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ast

## Preprocessing

### Load CSV

In [2]:
import keras
print(keras.__version__)

df = pd.read_csv("data/session-data.csv")
df

3.7.0


,index_x,SongID,TimeStamp_Central,Performer_x,Album,Song_x,TimeStamp_UTC,index_y,Performer_y,Song_y,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,session_3_hour,session_id
0,0,Twenty Five MilesEdwin Starr,5/25/2021 5:18:00 PM,Edwin Starr,25 Miles,Twenty Five Miles,2021-05-25 23:18:00,9761,Edwin Starr,Twenty Five Miles,...,1.0,0.0607,0.0595,0.000015,0.2240,0.964,124.567,4.0,2021-05-25 21:00:00,4332
1,1,Devil's EyesGreyhounds,5/25/2021 5:15:00 PM,Greyhounds,Change of Pace,Devil's Eyes,2021-05-25 23:15:00,206,Greyhounds,Devil's Eyes,...,0.0,0.0456,0.3540,0.000414,0.0974,0.858,113.236,4.0,2021-05-25 21:00:00,4332
2,2,Pussy and PizzaMurs,5/25/2021 5:12:00 PM,Murs,Have a Nice Life,Pussy and Pizza,2021-05-25 23:12:00,6404,Murs,Pussy and Pizza,...,1.0,0.0659,0.0708,0.000004,0.0780,0.381,93.991,4.0,2021-05-25 21:00:00,4332
3,8,Our Special PlaceThe Heavy,5/25/2021 4:46:00 PM,The Heavy,Great Vengeance and Furious Fire,Our Special Place,2021-05-25 22:46:00,6205,The Heavy,Our Special Place,...,1.0,0.0386,0.2720,0.003610,0.0991,0.939,193.996,4.0,2021-05-25 21:00:00,4332
4,10,Make Peace and be FreePerfect Confusion,5/25/2021 4:39:00 PM,Perfect Confusion,Perfect Confusion,Make Peace and be Free,2021-05-25 22:39:00,6051,Perfect Confusion,Make Peace and be Free,...,1.0,0.0315,0.0138,0.000017,0.0649,0.431,78.037,4.0,2021-05-25 21:00:00,4332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50013,62902,From Me To You - Remastered 2009The Beatles,1/1/2017 10:04:00 AM,The Beatles,Past Masters (Vols. 1 & 2 / Remastered),From Me To You - Remastered 2009,2017-01-01 16:04:00,5693,The Beatles,From Me To You - Remastered 2009,...,1.0,0.0309,0.6130,0.000000,0.2690,0.966,136.125,4.0,2017-01-01 15:00:00,0
50014,62903,And I Love Her - Remastered 2009The Beatles,1/1/2017 10:01:00 AM,The Beatles,A Hard Day's Night (Remastered),And I Love Her - Remastered 2009,2017-01-01 16:01:00,360,The Beatles,And I Love Her - Remastered 2009,...,0.0,0.0337,0.6400,0.000000,0.0681,0.636,113.312,4.0,2017-01-01 15:00:00,0
50015,62904,Ticket To Ride - Remastered 2009The Beatles,1/1/2017 9:58:00 AM,The Beatles,Help! (Remastered),Ticket To Ride - Remastered 2009,2017-01-01 15:58:00,9715,The Beatles,Ticket To Ride - Remastered 2009,...,1.0,0.0678,0.0457,0.000000,0.2330,0.749,123.419,4.0,2017-01-01 15:00:00,0
50016,62905,Come Together - Remastered 2009The Beatles,1/1/2017 9:54:00 AM,The Beatles,Abbey Road (Remastered),Come Together - Remastered 2009,2017-01-01 15:54:00,7425,The Beatles,Come Together - Remastered 2009,...,0.0,0.0393,0.0302,0.248000,0.0926,0.187,165.007,4.0,2017-01-01 15:00:00,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50018 entries, 0 to 50017
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index_x                    50018 non-null  int64  
 1   SongID                     50018 non-null  object 
 2   TimeStamp_Central          50018 non-null  object 
 3   Performer_x                50018 non-null  object 
 4   Album                      47890 non-null  object 
 5   Song_x                     50018 non-null  object 
 6   TimeStamp_UTC              50018 non-null  object 
 7   index_y                    50018 non-null  int64  
 8   Performer_y                50018 non-null  object 
 9   Song_y                     50018 non-null  object 
 10  spotify_genre              50018 non-null  object 
 11  spotify_track_id           50018 non-null  object 
 12  spotify_track_preview_url  36001 non-null  object 
 13  spotify_track_duration_ms  50018 non-null  flo

In [4]:
test_col_name = 'mode'
df.loc[:, test_col_name]

0        1.0
1        0.0
2        1.0
3        1.0
4        1.0
        ... 
50013    1.0
50014    0.0
50015    1.0
50016    0.0
50017    1.0
Name: mode, Length: 50018, dtype: float64

## Remove N.A.N data

In [5]:
# df_filtered = df[~df['danceability'].isna()]
# df_filtered.info()

### Prepare Tensorflow Datasets

In [6]:
import time
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import train_test_split

# Feature columns (as provided)
feature_columns = [
    'spotify_genre',
    'spotify_track_popularity',
    'danceability',
    'loudness',
    'acousticness',
    'instrumentalness',
    'tempo',
]

# Define the DataPreprocessor class
class DataPreprocessor:
    def __init__(self, df, feature_columns, batch_size=16, fixed_genre_size=10, train_size=0.8):
        """
        Initializes the data preprocessor with necessary parameters and preprocessing layers.
        
        Args:
            df (DataFrame): The input DataFrame containing session data.
            feature_columns (list): List of feature column names.
            batch_size (int): The batch size for dataset creation.
            fixed_genre_size (int): The fixed size for genre vectorization.
            train_size (float): Proportion of the data to use for training (between 0 and 1).
        """
        self.df = df
        self.feature_columns = feature_columns
        self.batch_size = batch_size
        self.fixed_genre_size = fixed_genre_size
        self.train_size = train_size

        # Split the dataset into training and testing datasets
        self.train_df, self.test_df = train_test_split(self.df, train_size=self.train_size, random_state=42)
        
        # Numeric feature preprocessing
        self.numeric_data = self.df[feature_columns[1:]].apply(pd.to_numeric, errors='coerce')
        self.mean_values = self.numeric_data.mean()
        self.std_values = self.numeric_data.std()

        # Initialize LabelEncoder for SongID and spotify_genre
        self.song_id_encoder = LabelEncoder()
        self.genre_encoder = LabelEncoder()

        # Extract unique SongIDs and genres
        unique_song_ids = self.df['SongID'].unique()
        all_genres = []
        for genre_str in self.df['spotify_genre']:
            try:
                genre_list = ast.literal_eval(genre_str)  # Safely parse the string into a list
                if isinstance(genre_list, list):
                    all_genres.extend(genre_list)
            except Exception as e:
                print(f"Error parsing genre: {e}")

        unique_genres = list(set(all_genres))

        # Fit the LabelEncoders on the data
        self.song_id_encoder.fit(unique_song_ids)
        self.genre_encoder.fit(unique_genres)

        self.items_size = len(self.song_id_encoder.classes_)  # Number of unique SongIDs
        self.genres_size = len(self.genre_encoder.classes_)  
        
        self.dataset = None

    def preprocess_song_id(self, song_id):
        """
        Encode the SongID using LabelEncoder.
        """
        return self.song_id_encoder.transform([song_id])[0]

    def clean_genre(self, value, default_value=0, dtype=tf.int32):
        """
        Clean and process the 'spotify_genre' feature.
        """
        if value is None or (isinstance(value, str) and not value.strip()):
            return np.full((self.fixed_genre_size,), default_value, dtype=dtype.as_numpy_dtype)

        try:
            genre_list = eval(value) if isinstance(value, str) else value
            if isinstance(genre_list, list):
                genre_encoded = self.genre_encoder.transform(genre_list)
            else:
                genre_encoded = self.genre_encoder.transform([value])
        except Exception:
            genre_encoded = self.genre_encoder.transform([value])

        # Pad or truncate to fixed size
        return np.pad(genre_encoded, (0, max(0, self.fixed_genre_size - len(genre_encoded))),
                      mode='constant')[:self.fixed_genre_size].astype(dtype.as_numpy_dtype)

    def clean_numeric_feature(self, value, default_value=0.0, feature_name="feature", mean=None, std=None):
        """
        Clean, process, and normalize numerical features using Z-score normalization.
        """
        if value is None or (isinstance(value, float) and np.isnan(value)):
            return default_value

        try:
            value = float(value)
            # Apply Z-score normalization if mean and std are provided
            if mean is not None and std is not None and std != 0:
                z_score_value = (value - mean) / std
                return z_score_value
            return value  # Return raw value if no normalization
        except ValueError:
            return default_value

    def create_session_dataset(self, session_df):
        """
        Create session dataset as a list of dictionaries for each session.
        """
        session_df = session_df.sort_values(by=['session_id', 'TimeStamp_UTC'])
        grouped = session_df.groupby('session_id')
        sessions_data = []
        for session_id, group in grouped:
            session_data = group.to_dict(orient='records')
            sessions_data.append(session_data)
        return sessions_data

    def preprocess_data(self, sessions, k=1):
        """
        Preprocess session data into TensorFlow dataset with split genre and features,
        filtering out sequences where the next item sequence length is not greater than 10.
        """
        item_sequences = []
        next_item_sequences = []
        genre_sequences = []
        feature_sequences = []
        processed_item_count = 0

        for idx, session in enumerate(sessions):
            session_item_sequences = []
            session_next_item_sequences = []
            session_genre_sequences = []
            session_feature_sequences = []

            for i in range(len(session) - 1):
                # Process items
                session_item_encoded = self.preprocess_song_id(session[i]['SongID'])
                next_session_item_encoded = self.preprocess_song_id(session[i + 1]['SongID'])
                session_item_sequences.append(session_item_encoded)
                session_next_item_sequences.append(next_session_item_encoded)

                # Process genre
                genre_cleaned = self.clean_genre(session[i].get('spotify_genre', None))
                session_genre_sequences.append(genre_cleaned)

                # Process numerical features
                numeric_features = []
                for col in self.feature_columns:
                    if col != 'spotify_genre':
                        mean = self.mean_values.get(col, None)
                        std = self.std_values.get(col, None)
                        cleaned_feature = self.clean_numeric_feature(session[i].get(col, None), mean=mean, std=std)
                        numeric_features.append(cleaned_feature)

                session_feature_sequences.append(numeric_features)

            # Filter out sessions where the next item sequence length is not greater than 10
            if len(session_next_item_sequences) > k:
                # Extend sequences only if the next item sequence length is greater than 10
                item_sequences.extend(session_item_sequences)
                next_item_sequences.extend(session_next_item_sequences)
                genre_sequences.extend(session_genre_sequences)
                feature_sequences.extend(session_feature_sequences)
                processed_item_count += len(session_item_sequences)

                print(f"Session {idx + 1} processed with {len(session_item_sequences)} items.")
            else:
                print(f"Session {idx + 1} skipped because next item sequence length is {len(session_next_item_sequences)}.")

        print(f"Total processed items: {processed_item_count}")

        # Convert to tensors
        item_sequences = tf.stack(item_sequences, axis=-1)
        next_item_sequences = tf.stack(next_item_sequences, axis=-1)
        genre_sequences_tensor = tf.constant(genre_sequences, dtype=tf.int32)
        feature_sequences_tensor = tf.constant(feature_sequences, dtype=tf.float32)

        # Create TensorFlow dataset
        dataset = tf.data.Dataset.from_tensor_slices({
            'item': item_sequences,
            'genre': genre_sequences_tensor,
            'features': feature_sequences_tensor,
            'next_item': next_item_sequences
        })

        return dataset

    def create_session_dataset_tensor(self, k):
        """
        Main function to create session dataset as tensors and return the dataset.
        """
        if self.dataset is not None:
            print("Dataset already created")
            return
        
        print("Creating session dataset")
        sessions_data = self.create_session_dataset(self.train_df)  # Use train data for training
        print("Creating tensor dataset")
        dataset = self.preprocess_data(sessions_data, k=k)

        # Shuffle and batch the training data
        dataset = dataset.shuffle(buffer_size=1024).batch(self.batch_size, drop_remainder=True)

        self.dataset = dataset
        return dataset

    def get_test_data(self, k):
        """
        Return preprocessed test dataset without shuffling.
        """
        sessions_data = self.create_session_dataset(self.test_df)
        dataset = self.preprocess_data(sessions_data, k)

        # Batch the test data without shuffling
        dataset = dataset.batch(self.batch_size, drop_remainder=True)

        return dataset

    def batch_timer(self, dataset):
        """
        Timer function to track the time taken for batch processing.
        """
        for batch in dataset:
            start_time = time.time()

            # Simulate processing (e.g., model training or data transformation)
            end_time = time.time()
            batch_time = end_time - start_time
            print(f"Batch processing time: {batch_time:.4f} seconds")



In [7]:
preprocessor = DataPreprocessor(df[:1000], feature_columns)

# Create the session dataset tensor
train_dataset = preprocessor.create_session_dataset_tensor(k=2)
test_dataset = preprocessor.get_test_data(k=2)

Creating session dataset
Creating tensor dataset
Session 1 processed with 4 items.
Session 2 processed with 6 items.
Session 3 processed with 18 items.
Session 4 skipped because next item sequence length is 1.
Session 5 processed with 3 items.
Session 6 processed with 10 items.
Session 7 processed with 22 items.
Session 8 processed with 9 items.
Session 9 skipped because next item sequence length is 2.
Session 10 processed with 9 items.
Session 11 processed with 8 items.
Session 12 processed with 9 items.
Session 13 processed with 12 items.
Session 14 processed with 11 items.
Session 15 processed with 15 items.
Session 16 skipped because next item sequence length is 2.
Session 17 processed with 6 items.
Session 18 processed with 14 items.
Session 19 skipped because next item sequence length is 2.
Session 20 processed with 8 items.
Session 21 processed with 9 items.
Session 22 processed with 15 items.
Session 23 processed with 6 items.
Session 24 processed with 6 items.
Session 25 proce

In [8]:
for batch in train_dataset.take(1):
    print("Items (SongID):", batch['item'].numpy())
    print("Genre:", batch['genre'].numpy())
    print("Features:", batch['features'].numpy())
    print("Next Items (Next SongID):", batch['next_item'].numpy())
    
for batch in test_dataset.take(1):
    print("Items (SongID):", batch['item'].numpy())
    print("Genre:", batch['genre'].numpy())
    print("Features:", batch['features'].numpy())
    print("Next Items (Next SongID):", batch['next_item'].numpy())

Items (SongID): [391 220 149 368  74 455 139  24 148 473 230 451 156 218 486 626]
Genre: [[ 12 227 247 262   0   0   0   0   0   0]
 [  4  92 186 207 259 290   0   0   0   0]
 [179   0   0   0   0   0   0   0   0   0]
 [ 28 174 176 268 314   0   0   0   0   0]
 [ 38 103 122 128 200 201 271 292 306   0]
 [148 150 205 246   0   0   0   0   0   0]
 [ 29  83 243 249 260 320   0   0   0   0]
 [  4  68 117 118 186 271 286 290 310   0]
 [  6  68 140 191 271   0   0   0   0   0]
 [179   0   0   0   0   0   0   0   0   0]
 [  6  38  68 140 191 256 271   0   0   0]
 [  6  21  68 140 166 254 256 271 303   0]
 [  6 142 186 271 290   0   0   0   0   0]
 [122 128 147 151 201 205 216 271 299   0]
 [122 128 147 151 201 205 216 271 299   0]
 [170   0   0   0   0   0   0   0   0   0]]
Features: [[ 0.92993677 -0.38409698  1.0024786  -0.84509045 -0.3540921   0.97593945]
 [ 0.18184407 -0.62183243 -1.7416495   0.93507767 -0.3540921  -2.0438871 ]
 [-0.8539766   1.0358906  -0.9859964   0.24242198 -0.35262948 

## Define Model class

In [9]:
class ItemEmbedding(Layer):
    def __init__(self, num_items, item_embed_dim):
        super(ItemEmbedding, self).__init__()
        
        self.item_embedding = Embedding(input_dim=num_items, output_dim=item_embed_dim, mask_zero=True, name='item_embedding')

    def call(self, items):
        # Embed items
        items_embedded = self.item_embedding(items)
        return items_embedded

class GRU4REC(Model):
    def __init__(self, rnn_params, genre_embed_dim, item_embed_dim, ffn1_units, feature_dense_units,  preprocessed_data:DataPreprocessor):
        super(GRU4REC, self).__init__()
        print(f"items size: {preprocessed_data.items_size}")
        print(f"genres size: {preprocessed_data.genres_size}")
        self.embedding = ItemEmbedding(preprocessed_data.items_size, item_embed_dim)
        
        # Genre embedding (only for genre, which is categorical and a string)
        self.genre_embedding = Embedding(input_dim=preprocessed_data.genres_size, output_dim=genre_embed_dim, mask_zero=True, name='genre_embedding')
        
        # RNN layers
        self.rnn_layers = []
        self.rnn_layers.append(GRU(**rnn_params[0], return_sequences=True, name='gru_0'))
        for i in range(1, len(rnn_params) - 1):
            self.rnn_layers.append(GRU(**rnn_params[i], return_sequences=True, name=f'gru_{i}'))
        self.rnn_layers.append(GRU(**rnn_params[-1], return_sequences=False, name=f"gru_{len(rnn_params)-1})"))

        self.concat = Concatenate(axis=-1, name='concat_1')
        self.batch_norm = BatchNormalization(name='batchnorm')

        # Feed-forward layers
        self.feature_dense = Dense(feature_dense_units, activation='relu', name='feature_dense')  # Dense layer for features (if required)
        self.ffn1 = Dense(ffn1_units, name='feed_forward_1')
        self.activation1 = LeakyReLU(alpha=0.2, name='freaky_relu')
        self.out = Dense(preprocessed_data.items_size, activation='softmax', name='output_layer')

    def call(self, inputs, training=False):
        """
        Forward pass for the GRU4REC model.
        :param inputs: Tuple (item_sequences, item_features, item_genres)
        :param training: Boolean indicating if the model is in training mode
        """
        
        item_sequences, item_features, item_genres = inputs

        # Embed items
        item_embedded = self.embedding(item_sequences)
        item_embedded = tf.expand_dims(item_embedded, axis=1)
        # Genre embedding
        genre_embedded = self.genre_embedding(item_genres)
        genre_embedded = tf.reduce_mean(genre_embedded, axis=1)
        genre_embedded = tf.expand_dims(genre_embedded, axis=1)
        
        # Feature transformation (features are passed directly as floats, so no embedding is needed)
        feature_transformed = self.feature_dense(item_features)
        feature_transformed = tf.expand_dims(feature_transformed, axis=1)

        # Pass through RNN layers
        x = item_embedded
        x = self.rnn_layers[0](x)
        for i in range(1, len(self.rnn_layers)):
            x = self.concat([item_embedded, x])  # Concatenate item embeddings with RNN outputs
            x = self.rnn_layers[i](x)

        # Concatenate RNN output with feature embeddings and genre embeddings
        x = tf.expand_dims(x, axis=1)
        x = self.concat([x, feature_transformed, genre_embedded])
        x = self.batch_norm(x)

        # Feed-forward layers
        x = self.ffn1(x)
        x = self.activation1(x)
        logits = self.out(x)  # Shape: (batch_size, num_items)

        # Generate the sequence of items (choose the item with the highest probability using argmax)
        predicted_items = tf.argmax(logits, axis=-1)  # (batch_size, sequence_length)

        return predicted_items, logits  # Return both predicted item indices and logits (probabilities)

## Training Loop

In [10]:
from keras.api.metrics import Recall

class RecallAtK(tf.keras.metrics.Metric):
    def __init__(self, k=10, name="recall_at_k", **kwargs):
        super(RecallAtK, self).__init__(name=name, **kwargs)
        self.k = k
        self.recall_at_k = Recall(top_k=self.k)

    def update_state(self, y_true, y_pred, sample_weight=None):
        """
        Update the state of the metric.
        """
        # Since y_true is a list of true items and y_pred are the predicted scores,
        # we need to calculate recall for top-k predicted items
        y_true = tf.cast(y_true, tf.int32)

        # Calculate the top-k predicted items
        top_k_preds = tf.argsort(y_pred, axis=-1, direction='DESCENDING')[:, :self.k]

        # Calculate recall by comparing true labels with the top-k predictions
        recall = tf.reduce_mean(tf.cast(tf.equal(y_true, top_k_preds), tf.float32), axis=-1)
        return recall

    def result(self):
        return self.recall_at_k.result()

    def reset_state(self):
        self.recall_at_k.reset_state()

def train_gru4rec(model, train_dataset, optimizer, loss_fn, epochs, k, val_dataset=None):
    # Create the RecallAtK metric
    recall_at_k = RecallAtK(k=k)
    
    # Training Loop
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Reset metrics for the epoch
        # recall_at_k.reset_state()
        epoch_loss = 0.0
        
        # Train the model using the dataset
        for step, batch in enumerate(train_dataset):
            item_sequences = batch['item']
            item_genres = batch['genre']
            item_features = batch['features']
            targets = batch['next_item']
            # print(f"item_sequences:", item_sequences)
            # print(f"item_genres: ", item_genres)
            # print(f"item_features:", item_features)
            
            with tf.GradientTape() as tape:
                # Forward pass: outputs is a tuple (predicted_items, logits)
                predicted_items, logits = model((item_sequences, item_features, item_genres), training=True)
                logits = tf.squeeze(logits, axis=1)
                # print(logits)  # Log logits for debugging
                
                # Compute the loss using logits (not predicted_items)
                loss = loss_fn(targets, logits)  # Use logits for loss calculation
                print(f"Epoch {epoch+1}, Step {step+1} Loss = {loss}")
                epoch_loss += loss.numpy()
            
            # Compute gradients and apply them
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            # Update Recall@k metric using logits
            # recall_at_k.update_state(targets, logits)  # Use logits for metric calculation

            # # Generate the predicted sequence (for Recall at k)
            # input_sequence = item_sequences
            # num_predictions = k  # Length of the sequence to predict

            # predicted_sequence = []
            # input_seq = input_sequence  # Start with the initial sequence

            # for _ in range(num_predictions):
            #     # Get the model's prediction (logits) for the next item
            #     predicted_items, logits = model(input_seq, training=False)

            #     # Add the predicted item (argmax) to the sequence
            #     predicted_sequence.append(predicted_items)

            #     # Update the input sequence by appending the predicted item
            #     input_seq = tf.concat([input_seq, predicted_items[:, -1:]], axis=-1)  # Append the last predicted item

            # predicted_sequence = tf.stack(predicted_sequence, axis=1)  # Shape: [batch_size, num_predictions]

            # # Calculate recall by comparing predicted sequence with targets
            # for batch_idx in range(predicted_sequence.shape[0]):
            #     top_k_preds = predicted_sequence[batch_idx]  # Predicted top-k items for this batch item
            #     true_item = targets[batch_idx]  # True next item for this batch item
                
            #     # Check if true item is in the top-k predictions
            #     if true_item in top_k_preds.numpy():
            #         recall_at_k.update_state(tf.convert_to_tensor([true_item]), predicted_sequence)  # Update recall metric
        
        # Calculate average loss and Recall@k for the training epoch
        avg_loss = epoch_loss / (step + 1)
        # train_recall_at_k = recall_at_k.result().numpy()
        print(f"Training loss: {avg_loss:.4f}")
        
        if val_dataset is None:
            continue
        
        # Validation step
        val_loss = 0.0
        
        for step, batch in enumerate(val_dataset):
            item_sequences = batch['item']
            item_genres = batch['genre']
            item_features = batch['features']
            targets = batch['next_item']
            # print(f"item_sequences:", item_sequences)
            # print(f"item_genres: ", item_genres)
            # print(f"item_features:", item_features)
            # Forward pass on validation set
            predicted_items, logits = model((item_sequences, item_features, item_genres), training=False)
            logits = tf.squeeze(logits, axis=1)
            loss = loss_fn(targets, logits)
            val_loss += loss.numpy()
            
            # # Update validation metrics
            # recall_at_k.update_state(targets, logits)
        
        avg_val_loss = val_loss / (step + 1)
        print(f"Validation loss: {avg_val_loss:.4f}")

def plot_training_history(loss_history, metric_history, metric_name, top_k):
    """Plot the training loss and accuracy."""
    epochs = range(1, len(loss_history) + 1)

    # Create subplots for loss and accuracy
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

    # Plot the training loss
    ax1.plot(epochs, loss_history, label='Loss', color='blue', linestyle='-', marker='o')
    ax1.set_title('Training Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Plot the top-k accuracy
    ax2.plot(epochs, metric_history, label=metric_name, color='green', linestyle='-', marker='o')
    ax2.set_title(f'Training {metric_name}')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel(metric_name)
    ax2.legend()

    plt.tight_layout()
    plt.show()

## Run the training process

In [11]:
# Define the model
# num_items = len()
# feature_vocab_size = len(feature_columns)

model = GRU4REC(
    rnn_params=[
        {"units": 128, "dropout": 0.3},
        {"units": 64, "dropout": 0.3}
    ],
    item_embed_dim=16,
    genre_embed_dim=16,
    ffn1_units=128,
    feature_dense_units=32,
    preprocessed_data=preprocessor
)

# model = GRU4REC(
#     rnn_params=[
#         {"units": 128},
#         {"units": 128},
#         {"units": 64}
#     ],
#     item_embed_dim=32,
#     genre_embed_dim=16,
#     ffn1_units=128,
#     feature_dense_units=64,
#     preprocessed_data=preprocessor
# )


items size: 635
genres size: 335


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [ ]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Train the model
train_gru4rec(model=model, train_dataset=train_dataset,optimizer=optimizer, loss_fn=loss_fn, epochs=500, k=8)

Epoch 1/50


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\layer.py:1381: UserWarning: Layer 'gru4rec' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: '''gru_1)' is not a valid root scope name. A root scope name has to match the following pattern: ^[A-Za-z0-9.][A-Za-z0-9_.\\/>-]*$''
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\layer.py:391: UserWarning: `build()` was 

Epoch 1, Step 1 Loss = 6.425483703613281
Epoch 1, Step 2 Loss = 6.569718837738037
Epoch 1, Step 3 Loss = 6.571135520935059
Epoch 1, Step 4 Loss = 6.483668804168701
Epoch 1, Step 5 Loss = 6.523014068603516
Epoch 1, Step 6 Loss = 6.4677042961120605
Epoch 1, Step 7 Loss = 6.403263092041016
Epoch 1, Step 8 Loss = 6.519145488739014
Epoch 1, Step 9 Loss = 6.466794967651367
Epoch 1, Step 10 Loss = 6.403201103210449
Epoch 1, Step 11 Loss = 6.408785820007324
Epoch 1, Step 12 Loss = 6.4136457443237305
Epoch 1, Step 13 Loss = 6.498261451721191
Epoch 1, Step 14 Loss = 6.5286078453063965
Epoch 1, Step 15 Loss = 6.523867607116699
Epoch 1, Step 16 Loss = 6.506232261657715
Epoch 1, Step 17 Loss = 6.59171199798584
Epoch 1, Step 18 Loss = 6.595395088195801
Epoch 1, Step 19 Loss = 6.615542411804199
Epoch 1, Step 20 Loss = 6.568951606750488
Epoch 1, Step 21 Loss = 6.4454545974731445
Epoch 1, Step 22 Loss = 6.6328558921813965
Epoch 1, Step 23 Loss = 6.5021257400512695
Epoch 1, Step 24 Loss = 6.454003810882

In [13]:
def predict(model, item_sequences, item_features, item_genres):
    """
    Predict the item with the highest probability for the given input sequences using argmax of softmax.

    Args:
    - model: The trained model.
    - item_sequences: Input item sequences (batch_size, seq_length).
    - item_features: Input item features (batch_size, feature_length).
    - item_genres: Input item genres (batch_size, genre_length).

    Returns:
    - predicted_items: A list of predicted items with the highest probability for each input sequence.
    """
    # Run the model in inference mode (not training)
    predicted_items, logits = model((item_sequences, item_features, item_genres), training=False)
    
    # Apply softmax to the logits to get probabilities
    probabilities = tf.nn.softmax(logits, axis=-1)
    
    # Get the item with the highest probability by finding the index of the maximum probability
    predicted_items = tf.argmax(probabilities, axis=-1, output_type=tf.int32)

    # Convert to numpy array for easier handling
    predicted_items = predicted_items.numpy()

    return predicted_items

def compute_recall(predicted_items, targets):
    """
    Compute the recall for the given predictions and targets.

    Args:
    - predicted_items: The predicted items (batch_size,).
    - targets: The actual next items (batch_size,).

    Returns:
    - recall: The recall metric.
    """
    # True positives: Predicted item matches the target
    true_positives = np.sum(predicted_items == targets)
    
    # Total relevant items (in this case, it is the number of items in the batch)
    total_items = len(targets)
    
    # Recall calculation
    recall = true_positives / total_items if total_items > 0 else 0
    return recall

# Initialize variables to calculate overall recall
total_true_positives = 0
total_items = 0

# Loop through training dataset and predict the most probable item
for step, batch in enumerate(train_dataset):
    item_sequences = batch['item']
    item_genres = batch['genre']
    item_features = batch['features']
    targets = batch['next_item']
    
    # Get the predicted item with the highest probability for each sequence in the batch
    predicted_items = predict(model, item_sequences, item_features, item_genres)
    
    # Compute recall for the current batch
    batch_recall = compute_recall(predicted_items, targets)
    print(f"Batch {step + 1} recall: {batch_recall:.4f}")
    
    # Accumulate for overall recall
    total_true_positives += np.sum(predicted_items == targets)
    total_items += len(targets)

# Calculate overall recall
overall_recall = total_true_positives / total_items if total_items > 0 else 0
print(f"Overall recall: {overall_recall:.4f}")


Batch 1 recall: 0.6875
Batch 2 recall: 0.4375
Batch 3 recall: 0.4375
Batch 4 recall: 0.8125
Batch 5 recall: 0.7500
Batch 6 recall: 0.7500
Batch 7 recall: 1.0000
Batch 8 recall: 0.7500
Batch 9 recall: 0.6875
Batch 10 recall: 0.6875
Batch 11 recall: 0.8125
Batch 12 recall: 0.8125
Batch 13 recall: 0.7500
Batch 14 recall: 0.7500
Batch 15 recall: 0.5000
Batch 16 recall: 0.9375
Batch 17 recall: 0.7500
Batch 18 recall: 0.6250
Batch 19 recall: 0.8750
Batch 20 recall: 0.6875
Batch 21 recall: 0.7500
Batch 22 recall: 0.8125
Batch 23 recall: 0.8750
Batch 24 recall: 0.6250
Batch 25 recall: 0.8750
Batch 26 recall: 0.9375
Batch 27 recall: 0.7500
Batch 28 recall: 0.7500
Batch 29 recall: 0.7500
Batch 30 recall: 0.8125
Batch 31 recall: 0.8125
Batch 32 recall: 0.8750
Batch 33 recall: 0.8750
Batch 34 recall: 0.8125
Batch 35 recall: 0.6875
Batch 36 recall: 0.8125
Batch 37 recall: 0.7500
Batch 38 recall: 0.8750
Batch 39 recall: 0.7500
Batch 40 recall: 0.6875
Batch 41 recall: 1.0000
Overall recall: 0.7652


In [14]:
model.save('gru4rec_model.h5', save_format='h5')